also try DeBERT

In [2]:
import pandas as pd

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

2024-08-30 11:25:28.124504: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Loading in DataFrames

In [3]:
filepath_labeled = "../data/processed/processed_for_finbert_labeled_data.csv"
filepath_unlabeled = "../data/processed/processed_for_finbert_spx500_unlabeled_data.csv"

In [4]:
df_labeled = pd.read_csv(filepath_labeled)
df_unlabeled = pd.read_csv(filepath_unlabeled)

In [10]:
# Define a mapping from sentiment labels to numerical values
label_map = {"negative": 0, "neutral": 1, "positive": 2}

# Apply the mapping to the 'sentiment' column in the labeled dataframe
df_labeled['encoded_sentiment'] = df_labeled['sentiment'].map(label_map)

In [11]:
df_labeled.head(3)

,newsHeadline,sentiment,processed_txt,encoded_sentiment
0,"According to Gran , the company has no plans t...",neutral,"According to Gran , the company has no plans t...",1
1,Technopolis plans to develop in stages an area...,neutral,Technopolis plans to develop in stages an area...,1
2,The international electronic industry company ...,negative,The international electronic industry company ...,0


In [6]:
df_unlabeled.head(3)

,newsHeadline,processed_txt
0,"Inflation data, retail sales, Walmart earnings...","Inflation data, retail sales, Walmart earnings..."
1,Whipsaw week for stocks leaves markets 'on edg...,Whipsaw week for stocks leaves markets 'on edg...
2,Down Between 12% and 24% From Their 52-Week Hi...,Down Between 12% and 24% From Their 52-Week Hi...


### Fine-tuning FinBERT

In [2]:
# Load pre-trained FinBERT
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

/Users/nikitapiko/Documents/University/WU Wien/SBWL/Data Science/dataenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [12]:
# Encode the labeled dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_labeled['processed_txt'].tolist(), 
    df_labeled['encoded_sentiment'].tolist(), 
    test_size=0.2, 
    random_state=42
)

In [17]:
# Set the maximum length to 512 tokens (standard for BERT-based models)
MAX_LENGTH = 512

In [18]:
# Encode the labeled dataset with truncation and padding
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LENGTH)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LENGTH)

In [19]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels  # These should be numerical
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels  # These should be numerical
})

In [22]:
# Define training arguments with logging
training_args = TrainingArguments(
    output_dir='./results',            # output directory
    num_train_epochs=3,                # number of training epochs
    per_device_train_batch_size=16,    # batch size per device during training
    per_device_eval_batch_size=16,     # batch size for evaluation
    warmup_steps=500,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir='./logs',              # directory for storing logs
    logging_steps=10,                  # log every 10 steps
    evaluation_strategy="steps",       # evaluate at each logging step
    eval_steps=50,                     # evaluate every 50 steps
    save_steps=100,                    # save checkpoint every 100 steps
    save_total_limit=2,                # limit total amount of checkpoints
    load_best_model_at_end=True,       # load the best model when finished training (great for early stopping)
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [24]:
# Train the model
trainer.train()

/Users/nikitapiko/Documents/University/WU Wien/SBWL/Data Science/dataenv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5688
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1068


Step,Training Loss,Validation Loss
50,3.391600,3.215135
100,0.958200,0.752151
150,0.650400,0.529844
200,0.453100,0.447301
250,0.488700,0.338261
300,0.457400,0.352095
350,0.309100,0.318879
400,0.199100,0.380845
450,0.373300,0.473788
500,0.252000,0.345929


***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-100
Configuration saved in ./results/checkpoint-100/config.json
Model weights saved in ./results/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1422
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-300
Configuration saved in ./results/checkpoint-300/config.json
Model weights saved in ./results/checkpoint-300/pytorch_model.bin
Deleting older checkpoint [results/check

TrainOutput(global_step=1068, training_loss=0.7119130513418034, metrics={'train_runtime': 12474.3657, 'train_samples_per_second': 1.368, 'train_steps_per_second': 0.086, 'total_flos': 1175056301278944.0, 'train_loss': 0.7119130513418034, 'epoch': 3.0})

In [28]:
df_unlabeled.head(3)

,newsHeadline,processed_txt
0,"Inflation data, retail sales, Walmart earnings...","Inflation data, retail sales, Walmart earnings..."
1,Whipsaw week for stocks leaves markets 'on edg...,Whipsaw week for stocks leaves markets 'on edg...
2,Down Between 12% and 24% From Their 52-Week Hi...,Down Between 12% and 24% From Their 52-Week Hi...


In [26]:
# Encode the unlabeled dataset with truncation and padding
unlabeled_encodings = tokenizer(df_unlabeled['processed_txt'].tolist(), truncation=True, padding=True, max_length=MAX_LENGTH)

unlabeled_dataset = Dataset.from_dict({
    'input_ids': unlabeled_encodings['input_ids'],
    'attention_mask': unlabeled_encodings['attention_mask']
})

In [29]:
# Predict sentiment using the fine-tuned model
predictions = trainer.predict(unlabeled_dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Map numerical labels back to sentiment labels
reverse_label_map = {0: "negative", 1: "neutral", 2: "positive"}
df_unlabeled['predicted_sentiment'] = [reverse_label_map[label] for label in predicted_labels]



***** Running Prediction *****
  Num examples = 198
  Batch size = 16


In [32]:
# Save the predictions to a CSV file
df_unlabeled[['processed_txt', 'predicted_sentiment']].to_csv('../data/unlabeled_with_predicted_sentiment.txt', index=False)

In [30]:
df_unlabeled.head()

,newsHeadline,processed_txt,predicted_sentiment
0,"Inflation data, retail sales, Walmart earnings...","Inflation data, retail sales, Walmart earnings...",positive
1,Whipsaw week for stocks leaves markets 'on edg...,Whipsaw week for stocks leaves markets 'on edg...,neutral
2,Down Between 12% and 24% From Their 52-Week Hi...,Down Between 12% and 24% From Their 52-Week Hi...,negative
3,Traders Bet on Wild Swings With CPI Print Set ...,Traders Bet on Wild Swings With CPI Print Set ...,neutral
4,"Down 60% This Year, Is Intel Stock a Bargain Buy?","Down 60% This Year, Is Intel Stock a Bargain Buy?",positive


### BONUS: Claude Experimentation

In [39]:
df_claude = pd.read_csv('../data/claude-stock-headlines-sentiment-analysis.csv')

In [40]:
df_claude.head()

,processed_txt,predicted_sentiment,claude_sentiment
0,"Inflation data, retail sales, Walmart earnings...",positive,neutral
1,Whipsaw week for stocks leaves markets 'on edg...,neutral,negative
2,Down Between 12% and 24% From Their 52-Week Hi...,negative,mixed
3,Traders Bet on Wild Swings With CPI Print Set ...,neutral,neutral
4,"Down 60% This Year, Is Intel Stock a Bargain Buy?",positive,mixed


In [45]:
df_claude['equal_or_not'] = 0

In [58]:
df_claude.loc[df_claude['predicted_sentiment'] == df_claude['claude_sentiment'], 'equal_or_not'] = 1

In [51]:
df_claude

,processed_txt,predicted_sentiment,claude_sentiment,equal_or_not
0,"Inflation data, retail sales, Walmart earnings...",positive,neutral,0
1,Whipsaw week for stocks leaves markets 'on edg...,neutral,negative,0
2,Down Between 12% and 24% From Their 52-Week Hi...,negative,mixed,0
3,Traders Bet on Wild Swings With CPI Print Set ...,neutral,neutral,1
4,"Down 60% This Year, Is Intel Stock a Bargain Buy?",positive,mixed,0
...,...,...,...,...
99,Buffett's Apple Share Dump Is Set to Reshape M...,negative,neutral,0
100,"Stock market today: S&P 500, Nasdaq rise, near...",positive,positive,1
101,Stocks Open Lower to Close Out a Volatile Trad...,negative,negative,1
102,"Tech stocks, bitcoin above $60k, Powell under ...",positive,mixed,0


In [52]:
df_claude.equal_or_not.value_counts()

equal_or_not
1    59
0    45
Name: count, dtype: int64

In [55]:
df_claude.loc[df_claude['claude_sentiment'] == 'mixed', 'claude_sentiment'] = df_claude.loc[df_claude['claude_sentiment'] == 'mixed', 'predicted_sentiment']

In [59]:
df_claude.equal_or_not.value_counts()

equal_or_not
1    81
0    23
Name: count, dtype: int64